# Date Format

In [ ]:
import tensorflow as tf

vocab_size = 10000
max_length = 50
embed_size = 128
num_heads = 5
ff_dim = 512


# Input layers
encoder_inputs = tf.keras.Input(shape=(None, ), dtype = tf.int32, name='encoder_inputs')
decoder_inputs = tf.keras.Input(shape=(None, ), dtype = tf.int32, name='decoder_inputs')


# Embedding layer
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                                    output_dim = embed_size,
                                                    mask_zero = True)


decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                                    output_dim = embed_size,
                                                    mask_zero = True)

encoder_embeddings = encoder_embedding_layer(encoder_inputs)
decoder_embeddings = decoder_embedding_layer(decoder_inputs)


# Positional Embedding
pos_embedding_layer = tf.keras.layers.Embedding(max_length, embed_size)
positions_encoder = tf.keras.layers.Lambda(lambda x: tf.range(start=0, limit = tf.shape(x)[1], delta=1))(encoder_inputs)
positions_decoder = tf.keras.layers.Lambda(lambda x: tf.range(start=0, limit = tf.shape(x)[1], delta=1))(decoder_inputs)
pos_embed_enc = pos_embedding_layer(positions_encoder)
pos_embed_dec = pos_embedding_layer(positions_decoder)



# Adding positions and token embeddings
encoder_embed = encoder_embeddings + pos_embed_enc
decoder_embed = decoder_embeddings + pos_embed_dec



# Encoder self-attention
encoder_attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size)(encoder_embed, encoder_embed)
encoder_attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoder_embed + encoder_attention)


# Encoder feed-forward
encoder_ff = tf.keras.layers.Dense(ff_dim, activation='relu')(encoder_attention)
encoder_ff = tf.keras.layers.Dense(embed_size)(encoder_ff)
encoder_outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoder_attention + encoder_ff)


# Decoder self-attention
causal_mask = tf.keras.layers.Lambda(
    lambda x: tf.linalg.band_part(tf.ones((tf.shape(x)[1], tf.shape(x)[1])), -1, 0)
)(decoder_inputs)

decoder_attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, \
                                                       key_dim=embed_size)(decoder_embed, decoder_embed, attention_mask=causal_mask)
decoder_attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(decoder_embed + decoder_attention)



# Encoder Decoder cross-attention
cross_attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size)(decoder_attention, encoder_outputs, encoder_outputs)
decoder_cross = tf.keras.layers.LayerNormalization(epsilon=1e-6)(decoder_attention + cross_attention)


# Decoder feed-forward
decoder_ff = tf.keras.layers.Dense(ff_dim, activation='relu')(decoder_cross)
decoder_ff = tf.keras.layers.Dense(embed_size)(decoder_ff)
decoder_outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(decoder_cross + decoder_ff)


# Final output-layer
output_logits = tf.keras.layers.Dense(vocab_size, activation='softmax')(decoder_outputs)
transformer = tf.keras.Model([encoder_inputs, decoder_inputs], output_logits)

In [ ]:
transformer.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='nadam',
    metrics=['accuracy']
)

In [ ]:
from datetime import date

# cannot use strftime()'s %B format since it depends on the locale
MONTHS = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]

def random_dates(n_dates):
    min_date = date(1000, 1, 1).toordinal()
    max_date = date(9999, 12, 31).toordinal()

    ordinals = np.random.randint(max_date - min_date, size=n_dates) + min_date
    dates = [date.fromordinal(ordinal) for ordinal in ordinals]

    x = [MONTHS[dt.month - 1] + " " + dt.strftime("%d, %Y") for dt in dates]
    y = [dt.isoformat() for dt in dates]
    return x, y

In [ ]:
np.random.seed(42)

n_dates = 3
x_example, y_example = random_dates(n_dates)
print("{:25s}{:25s}".format("Input", "Target"))
print("-" * 50)
for idx in range(n_dates):
    print("{:25s}{:25s}".format(x_example[idx], y_example[idx]))

Input                    Target                   
--------------------------------------------------
September 20, 7075       7075-09-20               
May 15, 8579             8579-05-15               
January 11, 7103         7103-01-11               


In [ ]:
INPUT_CHARS = "".join(sorted(set("".join(MONTHS) + "0123456789, ")))
INPUT_CHARS

' ,0123456789ADFJMNOSabceghilmnoprstuvy'

In [ ]:
OUTPUT_CHARS = "0123456789-"

In [ ]:
def date_str_to_ids(date_str, chars=INPUT_CHARS):
    return [chars.index(c) for c in date_str]

In [ ]:
date_str_to_ids(x_example[0], INPUT_CHARS)


[19, 23, 31, 34, 23, 28, 21, 23, 32, 0, 4, 2, 1, 0, 9, 2, 9, 7]

In [ ]:
x_example[0]

'September 20, 7075'

In [ ]:
date_str_to_ids(y_example[0], OUTPUT_CHARS)

[7, 0, 7, 5, 10, 0, 9, 10, 2, 0]

In [ ]:
def prepare_date_strs(date_strs, chars=INPUT_CHARS):
    X_ids = [date_str_to_ids(dt, chars) for dt in date_strs]
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor() # using 0 as the padding token ID

def create_dataset(n_dates):
    x, y = random_dates(n_dates)
    return prepare_date_strs(x, INPUT_CHARS), prepare_date_strs(y, OUTPUT_CHARS)

In [ ]:
np.random.seed(42)

X_train, Y_train = create_dataset(10000)
X_valid, Y_valid = create_dataset(2000)
X_test, Y_test = create_dataset(2000)

In [ ]:
Y_train[0]

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([ 8,  1,  8,  6, 11,  1, 10, 11,  3,  1], dtype=int32)>

In [ ]:
sos_id = len(OUTPUT_CHARS) + 1

def shifted_output_sequences(Y):
    sos_tokens = tf.fill(dims=(len(Y), 1), value=sos_id)
    return tf.concat([sos_tokens, Y[:, :-1]], axis=1)

X_train_decoder = shifted_output_sequences(Y_train)
X_valid_decoder = shifted_output_sequences(Y_valid)
X_test_decoder = shifted_output_sequences(Y_test)

In [ ]:
Y_train

<tf.Tensor: shape=(10000, 10), dtype=int32, numpy=
array([[ 8,  1,  8, ..., 11,  3,  1],
       [ 9,  6,  8, ..., 11,  2,  6],
       [ 8,  2,  1, ..., 11,  2,  2],
       ...,
       [10,  8,  7, ..., 11,  4,  1],
       [ 2,  2,  3, ..., 11,  3,  8],
       [ 8,  9,  4, ..., 11,  3, 10]], dtype=int32)>

In [ ]:
Y_train[0]

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([ 8,  1,  8,  6, 11,  1, 10, 11,  3,  1], dtype=int32)>

In [ ]:
X_train_decoder[0]

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([12,  8,  1,  8,  6, 11,  1, 10, 11,  3], dtype=int32)>

In [ ]:
X_train_decoder

<tf.Tensor: shape=(10000, 10), dtype=int32, numpy=
array([[12,  8,  1, ..., 10, 11,  3],
       [12,  9,  6, ...,  6, 11,  2],
       [12,  8,  2, ...,  2, 11,  2],
       ...,
       [12, 10,  8, ...,  2, 11,  4],
       [12,  2,  2, ...,  3, 11,  3],
       [12,  8,  9, ...,  8, 11,  3]], dtype=int32)>

In [ ]:
history = transformer.fit([X_train, X_train_decoder], Y_train, epochs=10,
                    validation_data=([X_valid, X_valid_decoder], Y_valid))

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 382s 1s/step - accuracy: 0.5984 - loss: 2.6371 - val_accuracy: 1.0000 - val_loss: 0.0084
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 303s 969ms/step - accuracy: 1.0000 - loss: 0.0060 - val_accuracy: 1.0000 - val_loss: 0.0024
Epoch 3/10
125/313 ━━━━━━━━━━━━━━━━━━━━ 2:27 784ms/step - accuracy: 1.0000 - loss: 0.0022

KeyboardInterrupt: 

In [ ]:
def ids_to_date_strs(ids, chars=OUTPUT_CHARS):
    return ["".join([("?" + chars)[index] for index in sequence])
            for sequence in ids]

In [ ]:
max_output_length = Y_train.shape[1]

In [ ]:
max_input_length = X_train.shape[1]

def prepare_date_strs_padded(date_strs):
    X = prepare_date_strs(date_strs)
    if X.shape[1] < max_input_length:
        X = tf.pad(X, [[0, 0], [0, max_input_length - X.shape[1]]])
    return X

def convert_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    ids = model.predict(X).argmax(axis=-1)
    return ids_to_date_strs(ids)

In [ ]:
sos_id = len(OUTPUT_CHARS) + 1

def predict_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    Y_pred = tf.fill(dims=(len(X), 1), value=sos_id)
    for index in range(max_output_length):
        pad_size = max_output_length - Y_pred.shape[1]
        X_decoder = tf.pad(Y_pred, [[0, 0], [0, pad_size]])
        Y_probas_next = transformer.predict([X, X_decoder])[:, index:index+1]
        Y_pred_next = tf.argmax(Y_probas_next, axis=-1, output_type=tf.int32)
        Y_pred = tf.concat([Y_pred, Y_pred_next], axis=1)
    return ids_to_date_strs(Y_pred[:, 1:])

In [ ]:
predict_date_strs(["July 14, 1789", \
                   "May 01, 2020", \
                   "September 11, 1245", \
                   "January 02, 7777"])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step


['1789-07-14', '2020-05-01', '1245-09-11', '7777-01-02']

In [ ]:
predict_date_strs(['gambling'])

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step


['7468-06-15']